In [2]:
import scib

In [3]:
scib.__version__

'1.1.5'

In [4]:
import scanpy as sc

In [5]:
adata_invae = sc.read_h5ad('/lustre/scratch126/cellgen/team205/dp26/Lung_5loc/invae/result/adata_train_Combat_processed.h5ad')
adata_scanvi = sc.read_h5ad("/lustre/scratch126/cellgen/team292/ha10/data/Spatial_Lung/adata_train_scANVI_processed.h5ad")
adata_scanvi = adata_scanvi[adata_invae.obs_names]
adata_invae.obsm['X_scANVI'] = adata_scanvi.obsm['X_scANVI']
adata_invae.obsm['X_scVI'] = adata_scanvi.obsm['X_scVI']
adata_scpoli = sc.read_h5ad('/lustre/scratch126/cellgen/team205/dp26/Lung_5loc/invae/result/adata_train_inVAE_scPoli_processed.h5ad')
adata_invae.obsm['X_scPoli'] = adata_scpoli.obsm['X_scPoli']

In [6]:
adata_invae.obs['Loc_celltype'] = adata_invae.obs['Loc_true'].astype(str) + '_' + adata_invae.obs['Celltypes_master_higher_immune'].astype(str)

In [7]:
adata_invae.obs['Loc_celltype'] = adata_invae.obs['Loc_celltype'].astype('category')

In [8]:
adata_invae

AnnData object with n_obs × n_vars = 46325 × 4000
    obs: 'Celltypes', 'Celltypes_master_high', 'Celltypes_master_higher', 'Celltypes_master_higher_immune', 'Loc_true', 'Material', 'Donor', 'Donor_ID_2', 'Gender', 'Sample', 'ID', 'Protocol_plot', 'Version', 'Study', 'PoolDon', 'DonorPool', 'scDonor_snBatch', 'n_counts', 'Loc_celltype'
    var: 'gene_ids', 'feature_types', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'Celltypes_colors', 'Donor_colors', 'Loc_true_colors', 'PoolDon_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_FinVAE', 'X_pca', 'X_pca_combat', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [9]:
sc.tl.pca(adata_invae)
sc.pp.neighbors(adata_invae, use_rep="X_pca")

In [10]:
adata_invae.write_h5ad("/lustre/scratch126/cellgen/team205/dp26/Lung_5loc/invae/result/adata_train_Combat_PCA_processed.h5ad")

In [9]:
sc.pp.neighbors(adata_invae, use_rep="X_FinVAE")
sc.tl.leiden(adata_invae,resolution=0.6)

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/umap/dist

In [14]:
results = {}

In [15]:
embed = 'X_FinVAE'

In [16]:
results[embed] = {}

In [20]:
results[embed]['KMeans NMI'] = scib.me.nmi(adata_invae, cluster_key="leiden", label_key="Loc_celltype")
results[embed]['KMeans ARI'] = scib.me.ari(adata_invae, cluster_key="leiden", label_key="Loc_celltype") 
results[embed]['Silhouette label'] = scib.me.silhouette(adata_invae, label_key="Loc_celltype", embed=embed, metric='euclidean', scale=True)
results[embed]['cLISI'] = scib.me.clisi_graph(adata_invae,batch_key="PoolDon", type_='knn', use_rep=embed,label_key="Loc_celltype")
results[embed]['Silhouette batch'] = scib.me.silhouette_batch(adata_invae, batch_key="PoolDon", label_key="Loc_celltype", embed=embed, metric='euclidean', return_all=False, scale=True, verbose=False)
results[embed]['Graph connectivity'] = scib.me.graph_connectivity(adata_invae, label_key="Loc_celltype")
results[embed]['Isolated labels'] = scib.me.isolated_labels(adata_invae, batch_key="PoolDon", embed=embed,label_key='Loc_true',iso_threshold=6,cluster_key='leiden',resolutions=[0.6])

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/scib-1.1.5-py3.8.egg/scib/metrics/lisi.py:172: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


In [24]:
sc.pp.neighbors(adata_invae, use_rep="X_scANVI")
sc.tl.leiden(adata_invae,resolution=0.6)

In [25]:
embed = 'X_scANVI'

In [26]:
results[embed] = {}

In [27]:
results[embed]['KMeans NMI'] = scib.me.nmi(adata_invae, cluster_key="leiden", label_key="Loc_celltype")
results[embed]['KMeans ARI'] = scib.me.ari(adata_invae, cluster_key="leiden", label_key="Loc_celltype") 
results[embed]['Silhouette label'] = scib.me.silhouette(adata_invae, label_key="Loc_celltype", embed=embed, metric='euclidean', scale=True)
results[embed]['cLISI'] = scib.me.clisi_graph(adata_invae,batch_key="PoolDon", type_='knn', use_rep=embed,label_key="Loc_celltype")
results[embed]['Silhouette batch'] = scib.me.silhouette_batch(adata_invae, batch_key="PoolDon", label_key="Loc_celltype", embed=embed, metric='euclidean', return_all=False, scale=True, verbose=False)
results[embed]['Graph connectivity'] = scib.me.graph_connectivity(adata_invae, label_key="Loc_celltype")
results[embed]['Isolated labels'] = scib.me.isolated_labels(adata_invae, batch_key="PoolDon", embed=embed,label_key='Loc_true',iso_threshold=6,cluster_key='leiden',resolutions=[0.6])

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/scib-1.1.5-py3.8.egg/scib/metrics/lisi.py:172: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


In [31]:
sc.pp.neighbors(adata_invae, use_rep="X_pca_combat")
sc.tl.leiden(adata_invae,resolution=0.6)

In [32]:
embed = 'X_pca_combat'

In [33]:
results[embed] = {}

In [34]:
results[embed]['KMeans NMI'] = scib.me.nmi(adata_invae, cluster_key="leiden", label_key="Loc_celltype")
results[embed]['KMeans ARI'] = scib.me.ari(adata_invae, cluster_key="leiden", label_key="Loc_celltype") 
results[embed]['Silhouette label'] = scib.me.silhouette(adata_invae, label_key="Loc_celltype", embed=embed, metric='euclidean', scale=True)
results[embed]['cLISI'] = scib.me.clisi_graph(adata_invae,batch_key="PoolDon", type_='knn', use_rep=embed,label_key="Loc_celltype")
results[embed]['Silhouette batch'] = scib.me.silhouette_batch(adata_invae, batch_key="PoolDon", label_key="Loc_celltype", embed=embed, metric='euclidean', return_all=False, scale=True, verbose=False)
results[embed]['Graph connectivity'] = scib.me.graph_connectivity(adata_invae, label_key="Loc_celltype")
results[embed]['Isolated labels'] = scib.me.isolated_labels(adata_invae, batch_key="PoolDon", embed=embed,label_key='Loc_true',iso_threshold=6,cluster_key='leiden',resolutions=[0.6])

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/scib-1.1.5-py3.8.egg/scib/metrics/lisi.py:172: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


In [39]:
sc.pp.neighbors(adata_invae, use_rep="X_scPoli")
sc.tl.leiden(adata_invae,resolution=0.6)

In [40]:
embed = 'X_scPoli'

In [41]:
results[embed] = {}

In [42]:
results[embed]['KMeans NMI'] = scib.me.nmi(adata_invae, cluster_key="leiden", label_key="Loc_celltype")
results[embed]['KMeans ARI'] = scib.me.ari(adata_invae, cluster_key="leiden", label_key="Loc_celltype") 
results[embed]['Silhouette label'] = scib.me.silhouette(adata_invae, label_key="Loc_celltype", embed=embed, metric='euclidean', scale=True)
results[embed]['cLISI'] = scib.me.clisi_graph(adata_invae,batch_key="PoolDon", type_='knn', use_rep=embed,label_key="Loc_celltype")
results[embed]['Silhouette batch'] = scib.me.silhouette_batch(adata_invae, batch_key="PoolDon", label_key="Loc_celltype", embed=embed, metric='euclidean', return_all=False, scale=True, verbose=False)
results[embed]['Graph connectivity'] = scib.me.graph_connectivity(adata_invae, label_key="Loc_celltype")
results[embed]['Isolated labels'] = scib.me.isolated_labels(adata_invae, batch_key="PoolDon", embed=embed,label_key='Loc_true',iso_threshold=6,cluster_key='leiden',resolutions=[0.6])

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/scib-1.1.5-py3.8.egg/scib/metrics/lisi.py:172: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


In [45]:
sc.pp.neighbors(adata_invae, use_rep="X_scVI")
sc.tl.leiden(adata_invae,resolution=0.6)

In [46]:
embed = 'X_scVI'

In [47]:
results[embed] = {}

In [48]:
results[embed]['KMeans NMI'] = scib.me.nmi(adata_invae, cluster_key="leiden", label_key="Loc_celltype")
results[embed]['KMeans ARI'] = scib.me.ari(adata_invae, cluster_key="leiden", label_key="Loc_celltype") 
results[embed]['Silhouette label'] = scib.me.silhouette(adata_invae, label_key="Loc_celltype", embed=embed, metric='euclidean', scale=True)
results[embed]['cLISI'] = scib.me.clisi_graph(adata_invae,batch_key="PoolDon", type_='knn', use_rep=embed,label_key="Loc_celltype")
results[embed]['Silhouette batch'] = scib.me.silhouette_batch(adata_invae, batch_key="PoolDon", label_key="Loc_celltype", embed=embed, metric='euclidean', return_all=False, scale=True, verbose=False)
results[embed]['Graph connectivity'] = scib.me.graph_connectivity(adata_invae, label_key="Loc_celltype")
results[embed]['Isolated labels'] = scib.me.isolated_labels(adata_invae, batch_key="PoolDon", embed=embed,label_key='Loc_true',iso_threshold=6,cluster_key='leiden',resolutions=[0.6])

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/scib-1.1.5-py3.8.egg/scib/metrics/lisi.py:172: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


In [11]:
sc.pp.neighbors(adata_invae, use_rep="X_pca")
sc.tl.leiden(adata_invae,resolution=0.6)

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/umap/dist

In [13]:
embed = 'X_pca'

In [14]:
results[embed] = {}

In [15]:
results[embed]['KMeans NMI'] = scib.me.nmi(adata_invae, cluster_key="leiden", label_key="Loc_celltype")
results[embed]['KMeans ARI'] = scib.me.ari(adata_invae, cluster_key="leiden", label_key="Loc_celltype") 
results[embed]['Silhouette label'] = scib.me.silhouette(adata_invae, label_key="Loc_celltype", embed=embed, metric='euclidean', scale=True)
results[embed]['cLISI'] = scib.me.clisi_graph(adata_invae,batch_key="PoolDon", type_='knn', use_rep=embed,label_key="Loc_celltype")
results[embed]['Silhouette batch'] = scib.me.silhouette_batch(adata_invae, batch_key="PoolDon", label_key="Loc_celltype", embed=embed, metric='euclidean', return_all=False, scale=True, verbose=False)
results[embed]['Graph connectivity'] = scib.me.graph_connectivity(adata_invae, label_key="Loc_celltype")
results[embed]['Isolated labels'] = scib.me.isolated_labels(adata_invae, batch_key="PoolDon", embed=embed,label_key='Loc_true',iso_threshold=6,cluster_key='leiden',resolutions=[0.6])

/home/jovyan/my-conda-envs/general/lib/python3.8/site-packages/scib-1.1.5-py3.8.egg/scib/metrics/lisi.py:172: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


In [52]:
import json
# Specify the file path where you want to save the JSON file
file_path = "scib_comparison_results.json"

# Save the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(results, json_file, indent=4) 